# Selector Group Chat

{py:class}`~autogen_agentchat.teams.SelectorGroupChat` implements a team where participants take turns broadcasting messages to all other participants,
with the next speaker selected by a generative model (e.g., an LLM) based on the shared context. 
This enables dynamic and context-aware multi-agent collaboration.

{py:class}`~autogen_agentchat.teams.SelectorGroupChat` provides several key features:
- Model-based speaker selection
- Configurable participant roles and descriptions
- Optional prevention of consecutive turns by the same speaker
- Customizable selection prompting
- Customizable selection function to override the default model-based selection

## How does it work?

{py:class}`~autogen_agentchat.teams.SelectorGroupChat` is a group chat similar to {py:class}`~autogen_agentchat.teams.RoundRobinGroupChat`,
but with a model-based next speaker selection mechanism.
When the team receives a task through {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run` or {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run_stream`,
the following steps are executed:

1. The team analyzes the current conversation context, including the conversation history and participants' {py:attr}`~autogen_agentchat.base.ChatAgent.name` and {py:attr}`~autogen_agentchat.base.ChatAgent.description` attributes, to determine the next speaker using a model.
2. The team prompts the selected speaker agent to provide a response, which is then broadcast to all other participants.
3. The termination condition is checked to determine if the conversation should end, if not, the process repeats from step 1.
4. When the conversation ends, the team returns the {py:class}`~autogen_agentchat.base.TaskResult` containing the conversation history from this task.

Once the team finishes the task, the conversation context is kept within the team and all participants, so the next task can continue from the previous conversation context.
You can reset the conversation context by calling {py:meth}`~autogen_agentchat.teams.BaseGroupChat.reset`.

## Task with Analysis Example

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.task import Console, MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models import OpenAIChatCompletionClient

### Agents

![Selector Group Chat](selector_group_chat.svg)

This system uses four specialized agents:

- **Planning Agent**: The strategic coordinator that breaks down complex tasks into manageable subtasks. 

- **Web Search Agent**: An information retrieval specialist that interfaces with the `search_web_tool`.
  
- **Data Analyst Agent**: An agent specialist in performing calculations equipped with `percentage_change_tool`. 

- **SelectorGroupChat Manager/Agent**: This one is not defined by the user, but is provided by the `SelectorGroupChat` class and orchestrates the conversation by selecting the next agent to handle the task.

In [3]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

In [8]:
model_client = OpenAIChatCompletionClient(model="gpt-4o")

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        Web search agent: Searches for information
        Data analyst: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="A web search agent.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="A data analyst agent. Useful for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    """,
)

### Workflow:
1. The task is received by the `SelectorGroupChat` manager which, based on agent descriptions, selects the most appropriate agent to handle the initial task (typically the Planning Agent).

2. The **Planning Agent** analyzes the task and breaks it down into subtasks, assigning each to the most appropriate agent using the format:
   `<agent> : <task>`

3. Based on the conversation context and agent descriptions, the `SelectorGroupChat` manager dynamically selects the next agent to handle their assigned subtask.

4. The **Web Search Agent** performs searches one at a time, storing results in the shared conversation history.

5. The **Data Analyst** processes the gathered information using available calculation tools when selected.

6. The workflow continues with agents being dynamically selected until either:
   - The Planning Agent determines all subtasks are complete and sends "TERMINATE"
   - An alternative termination condition is met (e.g., a maximum number of messages)

When defining your agents, make sure to include a helpful `description` since this is used by the `SelectorGroupChat` manager to decide which agent to select next.

In [9]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    termination_condition=termination,
)

task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

await Console(team.run_stream(task=task))

---------- user ----------

Who was the Miami Heat player with the highest points in the 2006-2007 season, 
and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- WebSearchAgent ----------
[FunctionCall(id='call_OwFkl5G65FQq052rDcMNE3aA', arguments='{"query": "Miami Heat highest points scorer 2006-2007 season"}', name='search_web_tool'), FunctionCall(id='call_qBrob8LwNP4hvPgiH0Dfjvvv', arguments='{"query": "Miami Heat player percentage change total rebounds 2007-2008 to 2008-2009"}', name='search_web_tool')]
[Prompt tokens: 144, Completion tokens: 76]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', call_id='call_OwFkl5G65FQq052rDcMNE3aA'), FunctionExecutionResult(content='The number of total rebounds fo

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='\nWho was the Miami Heat player with the highest points in the 2006-2007 season, \nand what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?'), ToolCallMessage(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=144, completion_tokens=76), content=[FunctionCall(id='call_OwFkl5G65FQq052rDcMNE3aA', arguments='{"query": "Miami Heat highest points scorer 2006-2007 season"}', name='search_web_tool'), FunctionCall(id='call_qBrob8LwNP4hvPgiH0Dfjvvv', arguments='{"query": "Miami Heat player percentage change total rebounds 2007-2008 to 2008-2009"}', name='search_web_tool')]), ToolCallResultMessage(source='WebSearchAgent', models_usage=None, content=[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n 

As we can see, after the Web Search Agent conducts the necessary searches and the Data Analyst Agent completes the necessary calculations, we find that Dwayne Wade was the Miami Heat player with the highest points in the 2006-2007 season, and the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons is 85.98%!